In [1]:
!python --version

Python 3.11.6


### 1. Ingestion Raw Data

In [17]:
# Connect postgres database
import psycopg2
import pandas as pd
import os

# Connect to your postgres DB
conn = psycopg2.connect(
    host=os.getenv("POSTGRES_HOST"),
    database=os.getenv("POSTGRES_DB"),
    user=os.getenv("POSTGRES_USER"),
    password=os.getenv("POSTGRES_PASSWORD")
)
conn.autocommit = True
cur = conn.cursor()
cur.execute("SELECT * FROM public.covid_data limit 1000000;")
df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

In [18]:
df.head()

,sno,observationdate,province_state,country_region,last_update,confirmed,deaths,recovered
0,108626,09/11/2020,Tabasco,Mexico,2021-04-02 15:13:53,29814.0,2689.0,26105.0
1,108627,09/11/2020,Tacna,Peru,2021-04-02 15:13:53,10120.0,174.0,0.0
2,108628,09/11/2020,Tamaulipas,Mexico,2021-04-02 15:13:53,26507.0,1951.0,23349.0
3,108629,09/11/2020,Tambov Oblast,Russia,2021-04-02 15:13:53,7039.0,46.0,6198.0
4,108630,09/11/2020,Tamil Nadu,India,2021-04-02 15:13:53,491571.0,8231.0,435422.0


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   sno              1000000 non-null  object
 1   observationdate  1000000 non-null  object
 2   province_state   1000000 non-null  object
 3   country_region   1000000 non-null  object
 4   last_update      1000000 non-null  object
 5   confirmed        1000000 non-null  object
 6   deaths           1000000 non-null  object
 7   recovered        1000000 non-null  object
dtypes: object(8)
memory usage: 61.0+ MB


In [20]:
df["sno"].duplicated().sum()

np.int64(693571)

### 2. Staging & Standardization

#### Normalize time

- จัดการกับ format time ให้อยู่ในรูปแบบเดียวกัน

In [5]:
import pandas as pd
from dateutil import parser

def normalize_datetime(series):
    """
    แปลง datetime หลาย format ให้เป็น 'YYYY-MM-DD HH:MM'
    """
    def parse_value(x):
        try:
            if pd.isna(x):
                return pd.NaT
            dt = parser.parse(str(x))
            return dt.strftime("%Y-%m-%d %H:%M")  # normalize format
        except Exception:
            return pd.NaT
    
    return series.apply(parse_value)


In [6]:
# แปลง formatข้อมูล datetime ให้เป็น 'YYYY-MM-DD HH:MM'
df["last_update"] = normalize_datetime(df["last_update"])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225716 entries, 0 to 1225715
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   sno              1225716 non-null  object
 1   observationdate  1225716 non-null  object
 2   province_state   1225716 non-null  object
 3   country_region   1225716 non-null  object
 4   last_update      1225716 non-null  object
 5   confirmed        1225716 non-null  object
 6   deaths           1225716 non-null  object
 7   recovered        1225716 non-null  object
dtypes: object(8)
memory usage: 74.8+ MB


In [8]:
df["last_update"].unique()[:10]

array(['2020-01-22 17:00', '2020-01-23 17:00', '2020-01-24 17:00',
       '2020-01-25 17:00', '2020-01-26 16:00', '2020-01-27 23:59',
       '2020-01-28 23:00', '2020-01-29 19:30', '2020-01-30 16:00',
       '2020-01-31 23:59'], dtype=object)

In [9]:
df["observationdate"].unique()[:10]

array(['01/22/2020', '01/23/2020', '01/24/2020', '01/25/2020',
       '01/26/2020', '01/27/2020', '01/28/2020', '01/29/2020',
       '01/30/2020', '01/31/2020'], dtype=object)

### 3. Rename columns

In [10]:
# ฟังก์ชันแปลงเป็น snake_case
def to_snake_case(col_name):
    # แปลง space, dash เป็น underscore
    col_name = re.sub(r"[ -]+", "_", col_name)
    # ลบตัวอักษรพิเศษ
    col_name = re.sub(r"[^\w_]", "", col_name)
    # แปลงเป็นตัวเล็ก
    return col_name.lower()

In [11]:
import re
df.columns = [to_snake_case(col) for col in df.columns]
df.head()

,sno,observationdate,province_state,country_region,last_update,confirmed,deaths,recovered
0,1,01/22/2020,Anhui,Mainland China,2020-01-22 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,2020-01-22 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,2020-01-22 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,2020-01-22 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,2020-01-22 17:00,0.0,0.0,0.0


### 3. Cast Data Type

In [ ]:
# แปลงชนิดข้อมูล
df.astype({
    "sno": "int",
    "province_state": "string",
    "country_region": "string"
    })

# แปลงวันที่
df["observationdate"] = pd.to_datetime(df["observationdate"], format="%m/%d/%Y")
df["last_update"] = pd.to_datetime(df["last_update"], format="%Y-%m-%d %H:%M")

# แปลงตัวเลข
df["confirmed"] = pd.to_numeric(df["confirmed"], errors="coerce").astype(int)
df["deaths"] = pd.to_numeric(df["deaths"], errors="coerce").astype(int)
df["recovered"] = pd.to_numeric(df["recovered"], errors="coerce").astype(int)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225716 entries, 0 to 1225715
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   sno              1225716 non-null  object        
 1   observationdate  1225716 non-null  datetime64[ns]
 2   province_state   1225716 non-null  object        
 3   country_region   1225716 non-null  object        
 4   last_update      1225716 non-null  datetime64[ns]
 5   confirmed        1225716 non-null  int64         
 6   deaths           1225716 non-null  int64         
 7   recovered        1225716 non-null  int64         
dtypes: datetime64[ns](2), int64(3), object(3)
memory usage: 74.8+ MB


In [14]:
df.isna().sum()

sno                0
observationdate    0
province_state     0
country_region     0
last_update        0
confirmed          0
deaths             0
recovered          0
dtype: int64

### 4. Handle Missing Value

- จัดการกับ missing value ดังนี้
    
    - แทนที่ด้วย ค่า mean (int) 
    
    - แทนที่ด้วย Unknow (str)

    - ลบ rows นั้นถ้าเป็น NaT (datetime)

In [15]:
# จัดการกับ missing value ดังนี้

# แทนที่ด้วย ค่า mean (int)
df["confirmed"] = df["confirmed"].fillna(int(df["confirmed"].mean()))
df["deaths"] = df["deaths"].fillna(int(df["deaths"].mean()))
df["recovered"] = df["recovered"].fillna(int(df["recovered"].mean()))

# แทนที่ด้วย Unknow (str)
df["province_state"] = df["province_state"].fillna("Unknown")
df["country_region"] = df["country_region"].fillna("Unknown")

# ลบ rows นั้นถ้าเป็น NaT (datetime)
df = df.dropna(subset=["observationdate", "last_update"])

In [16]:
df.isna().sum()

sno                0
observationdate    0
province_state     0
country_region     0
last_update        0
confirmed          0
deaths             0
recovered          0
dtype: int64

In [32]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225716 entries, 0 to 1225715
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   sno              1225716 non-null  object        
 1   observationdate  1225716 non-null  datetime64[ns]
 2   province_state   1225716 non-null  object        
 3   country_region   1225716 non-null  object        
 4   last_update      1225716 non-null  datetime64[ns]
 5   confirmed        1225716 non-null  int64         
 6   deaths           1225716 non-null  int64         
 7   recovered        1225716 non-null  int64         
dtypes: datetime64[ns](2), int64(3), object(3)
memory usage: 74.8+ MB


,sno,observationdate,province_state,country_region,last_update,confirmed,deaths,recovered
0,1,2020-01-22,Anhui,Mainland China,2020-01-22 17:00:00,1,0,0
1,2,2020-01-22,Beijing,Mainland China,2020-01-22 17:00:00,14,0,0
2,3,2020-01-22,Chongqing,Mainland China,2020-01-22 17:00:00,6,0,0
3,4,2020-01-22,Fujian,Mainland China,2020-01-22 17:00:00,1,0,0
4,5,2020-01-22,Gansu,Mainland China,2020-01-22 17:00:00,0,0,0


### 5. Save Data

In [31]:
df.to_csv("/home/jovyan/work/output/covid_data_cleaned.csv", index=False)